# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [157]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import seaborn as sn
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
og_df = pd.read_csv('city_data.csv')
og_df = og_df.drop(labels='Unnamed: 0', axis = 1)
clean_df = og_df.dropna(how='any')
clean_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.29,-156.79,30.20,86.0,90.0,17.22,US,1.603596e+09
1,albany,42.60,-73.97,46.00,68.0,61.0,1.99,US,1.603596e+09
2,rikitea,-23.12,-134.97,74.88,82.0,97.0,20.71,PF,1.603596e+09
3,kandalaksha,67.16,32.41,27.88,97.0,92.0,4.36,RU,1.603596e+09
4,atuona,-9.80,-139.03,78.66,75.0,30.0,20.56,PF,1.603596e+09
...,...,...,...,...,...,...,...,...,...
624,channel-port aux basques,47.57,-59.14,50.18,96.0,100.0,8.46,CA,1.603596e+09
625,nhulunbuy,-12.23,136.77,87.80,58.0,40.0,10.29,AU,1.603596e+09
626,atbasar,51.80,68.33,39.60,92.0,100.0,19.84,KZ,1.603596e+09
627,mildura,-34.20,142.15,61.00,31.0,40.0,13.87,AU,1.603596e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = clean_df[["Lat", "Lon"]].astype(float)

# Convert to float and store
humid_values = clean_df["Humidity"].astype(float)


In [32]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid_values, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [73]:
#was getting errors about conflicting data types when trying to narrow down the cities to fit weather conditions
#did some conversions and it started working

fake_df = clean_df.convert_dtypes()
fake_df['Wind Speed'] = clean_df['Wind Speed'].astype(np.int64)
#fake_df['Wind Speed'].dtype
#fake_df

In [74]:
#Joe's hint
#df_new =  df[(df[] < 90) & (df[] < 100) & (df[] < 10) & ....] 
paradise_df = fake_df[(fake_df['Wind Speed'] < 10) & 
                       (fake_df['Cloudiness']==0) & 
                       ((fake_df['Max Temp']>70) & (fake_df['Max Temp'] <= 80))]
paradise_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,arlit,18.74,7.39,75.00,18,0,9,NE,1603595626
18,livramento,-3.03,-60.18,77.00,100,0,2,BR,1603596142
20,cayenne,4.93,-52.33,77.00,94,0,1,GF,1603596142
27,tessalit,20.20,1.01,77.41,17,0,7,ML,1603595621
40,nizwa,22.93,57.53,70.18,23,0,7,OM,1603596145
94,saint-pierre,-21.34,55.48,73.40,43,0,5,RE,1603596152
281,saint-philippe,-21.36,55.77,73.40,43,0,5,RE,1603595892
302,kidal,18.44,1.41,75.25,18,0,9,ML,1603596180
434,khipro,25.83,69.38,76.59,41,0,7,PK,1603596199
451,parana,-31.73,-60.52,77.00,84,0,9,AR,1603595964


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [106]:
hotel_df = paradise_df
#hotel_df['Hotel Name'] = None
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,arlit,18.74,7.39,75.00,18,0,9,NE,1603595626,None
18,livramento,-3.03,-60.18,77.00,100,0,2,BR,1603596142,None
20,cayenne,4.93,-52.33,77.00,94,0,1,GF,1603596142,None
27,tessalit,20.20,1.01,77.41,17,0,7,ML,1603595621,None
40,nizwa,22.93,57.53,70.18,23,0,7,OM,1603596145,None
94,saint-pierre,-21.34,55.48,73.40,43,0,5,RE,1603596152,None
281,saint-philippe,-21.36,55.77,73.40,43,0,5,RE,1603595892,None
302,kidal,18.44,1.41,75.25,18,0,9,ML,1603596180,None
434,khipro,25.83,69.38,76.59,41,0,7,PK,1603596199,None
451,parana,-31.73,-60.52,77.00,84,0,9,AR,1603595964,None


In [137]:
#testing api calls
params = {
    "key": g_key,
    "input": 'cayenne',
    "inputtype": 'textquery'
}
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

response = requests.get(base_url, params=params).json()
results = response
print(results['candidates'][0]['place_id'])

ChIJnbdK2zo9Eo0RKLaviyl-VnM


In [167]:
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    x = hotel_df.loc[index, 'Lat']
    y = hotel_df.loc[index, 'Lon']
    print(f"Lat: {x}")
    print(f"Lon: {y}")

Lat: 18.74
Lon: 7.39
Lat: -3.03
Lon: -60.18
Lat: 4.93
Lon: -52.33
Lat: 20.2
Lon: 1.01
Lat: 22.93
Lon: 57.53
Lat: -21.34
Lon: 55.48
Lat: -21.36
Lon: 55.77
Lat: 18.44
Lon: 1.41
Lat: 25.83
Lon: 69.38
Lat: -31.73
Lon: -60.52
Lat: 22.69
Lon: 58.53
Lat: -21.15
Lon: 55.28
Lat: 25.29
Lon: 51.53


In [ ]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,    
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    x = hotel_df.loc[index, 'Lat'].astype(str)
    y = hotel_df.loc[index, 'Lon'].astype(str)
    
    # lat,lon from above
    params["location"] = [x + ',' + y]   
    print(params['location'])
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Use the search term: "Hotel" and our lat/lng
    params['keyword'] = 'Hotel'
    
    # make request and print url
    response = requests.get(base_url, params=params).json()
    
    pprint(response)
    # Grab the first hotel from the results and store the name
    #try:
        #hotel_df.loc[index, 'Hotel Name'] = response["results"][0]['name']
    #except(KeyError,IndexError):
        #print('Hotel not found... Skipping...')


['18.74,7.39']
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 18.7389762,
                                        'lng': 7.389448199999999},
                           'viewport': {'northeast': {'lat': 18.74026762989272,
                                                      'lng': 7.390824529892723},
                                        'southwest': {'lat': 18.73756797010728,
                                                      'lng': 7.388124870107279}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Hôtel Telwa Bungalow',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 2988,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/111080983184893115512">ismael '
                                            

In [181]:
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,arlit,18.74,7.39,75.00,18,0,9,NE,1603595626,None
18,livramento,-3.03,-60.18,77.00,100,0,2,BR,1603596142,None
20,cayenne,4.93,-52.33,77.00,94,0,1,GF,1603596142,None
27,tessalit,20.20,1.01,77.41,17,0,7,ML,1603595621,None
40,nizwa,22.93,57.53,70.18,23,0,7,OM,1603596145,None
94,saint-pierre,-21.34,55.48,73.40,43,0,5,RE,1603596152,None
281,saint-philippe,-21.36,55.77,73.40,43,0,5,RE,1603595892,None
302,kidal,18.44,1.41,75.25,18,0,9,ML,1603596180,None
434,khipro,25.83,69.38,76.59,41,0,7,PK,1603596199,None
451,parana,-31.73,-60.52,77.00,84,0,9,AR,1603595964,None


In [162]:
pprint(response['results'][0]['name'])

'Hôtel Telwa Bungalow'


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
